# 🏆 SÉLECTION DU MODÈLE FINAL
## Comparaison des performances des modèles entraînés

Ce notebook compare les trois modèles entraînés pour prédire le **rendement du maïs (yield)** :
1. Ridge Regression (Régression Linéaire Régularisée)
2. Random Forest Regressor
3. Gradient Boosting Regressor

In [ ]:
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

: 

## 1. Chargement des données et préparation

In [ ]:
# Chargement des données
df = pd.read_csv('../data/hvstat_africa_data_v1.0.csv')
df_maize = df[df['product'].str.contains('maize|corn|maïs', case=False, na=False)].copy()

# Nettoyage des outliers
mask = (
    df_maize['area'].between(0.1, 50000) &
    df_maize['production'].between(1, 50000) &
    df_maize['yield'].between(0.1, 8)
)
df_clean = df_maize[mask].copy()
df_clean = df_clean.dropna(subset=['yield', 'area', 'production'])

# Standardisation système de production
df_clean['system_simplified'] = 'other'
sys_lower = df_clean['crop_production_system'].str.lower()
df_clean.loc[sys_lower.str.contains('irrigated|water|dam|riverine', na=False), 'system_simplified'] = 'irrigated'
df_clean.loc[sys_lower.str.contains('rainfed|dieri|recessional', na=False), 'system_simplified'] = 'rainfed'
df_clean.loc[sys_lower.str.contains('commercial|mechanized|large_scale|lscf', na=False), 'system_simplified'] = 'commercial_mechanized'
df_clean.loc[sys_lower.str.contains('traditional|communal|small|pastoral|sscf|a1|a2', na=False), 'system_simplified'] = 'traditional_small_scale'
df_clean.loc[sys_lower.str.contains('all|none|or \(ps\)', na=False), 'system_simplified'] = 'general_unknown'

print(f"✅ Dataset nettoyé: {len(df_clean):,} observations")

In [ ]:
# Préparation des features
features_cols = ['country_code', 'season_name', 'planting_month', 'harvest_month', 'area', 'system_simplified']
X = df_clean[features_cols]
y = df_clean['yield']

# Encodage One-Hot
X_encoded = pd.get_dummies(X, columns=['country_code', 'season_name', 'system_simplified'], drop_first=True)

# Division Train/Test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Scaling pour Ridge Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Train: {X_train.shape[0]} samples, Test: {X_test.shape[0]} samples")
print(f"Features: {X_train.shape[1]} colonnes")

## 2. Chargement des modèles entraînés

In [ ]:
# Chargement des modèles
ridge_model = joblib.load('../ml_models_pkg/ridge_regression_model.pkl')
rf_model = joblib.load('../ml_models_pkg/random_forest_model.pkl')
gb_model = joblib.load('../ml_models_pkg/gb_model.pkl')

print("✅ Modèles chargés avec succès!")
print(f"  - Ridge Regression: alpha = {ridge_model.alpha}")
print(f"  - Random Forest: {rf_model.n_estimators} arbres, max_depth = {rf_model.max_depth}")
print(f"  - Gradient Boosting: {gb_model.n_estimators} arbres, learning_rate = {gb_model.learning_rate}")

## 3. Évaluation comparative des modèles

In [ ]:
def evaluate_model(model, X_test, y_test, model_name, scaled=False):
    """Évalue un modèle et retourne les métriques."""
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    return {
        'Modèle': model_name,
        'MAE (t/ha)': mae,
        'RMSE (t/ha)': rmse,
        'R² Score': r2,
        'Predictions': y_pred
    }

# Évaluation des trois modèles
results = []
results.append(evaluate_model(ridge_model, X_test_scaled, y_test, 'Ridge Regression'))
results.append(evaluate_model(rf_model, X_test, y_test, 'Random Forest'))
results.append(evaluate_model(gb_model, X_test, y_test, 'Gradient Boosting'))

# Création du tableau comparatif
comparison_df = pd.DataFrame([{k: v for k, v in r.items() if k != 'Predictions'} for r in results])
comparison_df = comparison_df.sort_values('R² Score', ascending=False)

print("\n" + "="*70)
print("📊 TABLEAU COMPARATIF DES PERFORMANCES")
print("="*70)
print(comparison_df.to_string(index=False))
print("="*70)

In [ ]:
# Visualisation des performances
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

colors = ['#3498db', '#2ecc71', '#e74c3c']
models = ['Ridge Regression', 'Random Forest', 'Gradient Boosting']

# R² Score
r2_scores = [r['R² Score'] for r in results]
bars1 = axes[0].bar(models, r2_scores, color=colors, edgecolor='black')
axes[0].set_ylabel('R² Score')
axes[0].set_title('R² Score (Plus élevé = Meilleur)', fontweight='bold')
axes[0].set_ylim(0, 1)
for bar, score in zip(bars1, r2_scores):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                 f'{score:.4f}', ha='center', fontweight='bold')

# MAE
mae_scores = [r['MAE (t/ha)'] for r in results]
bars2 = axes[1].bar(models, mae_scores, color=colors, edgecolor='black')
axes[1].set_ylabel('MAE (tonnes/ha)')
axes[1].set_title('MAE - Erreur Absolue Moyenne (Plus bas = Meilleur)', fontweight='bold')
for bar, score in zip(bars2, mae_scores):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                 f'{score:.4f}', ha='center', fontweight='bold')

# RMSE
rmse_scores = [r['RMSE (t/ha)'] for r in results]
bars3 = axes[2].bar(models, rmse_scores, color=colors, edgecolor='black')
axes[2].set_ylabel('RMSE (tonnes/ha)')
axes[2].set_title('RMSE - Racine de l\'Erreur Quadratique (Plus bas = Meilleur)', fontweight='bold')
for bar, score in zip(bars3, rmse_scores):
    axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
                 f'{score:.4f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../ml_models_pkg/model_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("\n📈 Graphique sauvegardé: ml_models_pkg/model_comparison.png")

## 4. Analyse des prédictions vs valeurs réelles

In [ ]:
# Graphiques de prédictions vs réalité
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for i, (result, ax, color) in enumerate(zip(results, axes, colors)):
    y_pred = result['Predictions']
    ax.scatter(y_test, y_pred, alpha=0.5, color=color, edgecolor='none', s=20)
    
    # Ligne parfaite (y = x)
    min_val, max_val = min(y_test.min(), y_pred.min()), max(y_test.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'k--', lw=2, label='Prédiction parfaite')
    
    ax.set_xlabel('Valeurs réelles (t/ha)')
    ax.set_ylabel('Prédictions (t/ha)')
    ax.set_title(f"{result['Modèle']}\nR² = {result['R² Score']:.4f}", fontweight='bold')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../ml_models_pkg/predictions_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Distribution des erreurs

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for i, (result, ax, color) in enumerate(zip(results, axes, colors)):
    y_pred = result['Predictions']
    errors = y_test.values - y_pred
    
    ax.hist(errors, bins=50, color=color, edgecolor='black', alpha=0.7)
    ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='Erreur nulle')
    ax.axvline(x=errors.mean(), color='blue', linestyle='-', linewidth=2, label=f'Moyenne: {errors.mean():.3f}')
    
    ax.set_xlabel('Erreur (t/ha)')
    ax.set_ylabel('Fréquence')
    ax.set_title(f"Distribution des erreurs - {result['Modèle']}", fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../ml_models_pkg/error_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. 🏆 SÉLECTION DU MODÈLE FINAL

In [ ]:
# Détermination du meilleur modèle
best_idx = np.argmax([r['R² Score'] for r in results])
best_model_name = results[best_idx]['Modèle']

print("\n" + "="*70)
print("🏆 MODÈLE FINAL SÉLECTIONNÉ")
print("="*70)
print(f"\n✅ Le modèle retenu est: {best_model_name}")
print(f"\n📊 Performances sur le jeu de test:")
print(f"   • R² Score: {results[best_idx]['R² Score']:.4f}")
print(f"   • MAE: {results[best_idx]['MAE (t/ha)']:.4f} t/ha")
print(f"   • RMSE: {results[best_idx]['RMSE (t/ha)']:.4f} t/ha")
print("\n" + "="*70)

In [ ]:
# Justification du choix
print("\n📝 JUSTIFICATION DU CHOIX DU MODÈLE")
print("-" * 50)

# Analyse comparative
ridge_r2 = results[0]['R² Score']
rf_r2 = results[1]['R² Score']
gb_r2 = results[2]['R² Score']

print(f"""
1. **Ridge Regression** (R² = {ridge_r2:.4f})
   - ✅ Simple et interprétable
   - ✅ Rapide à entraîner
   - ❌ Performance limitée sur données non-linéaires

2. **Random Forest** (R² = {rf_r2:.4f})
   - ✅ Robuste aux outliers
   - ✅ Capture les relations non-linéaires
   - ✅ Feature importance disponible
   - ❌ Peut être lent pour de gros datasets

3. **Gradient Boosting** (R² = {gb_r2:.4f})
   - ✅ Souvent le plus précis
   - ✅ Optimise itérativement les erreurs
   - ❌ Plus sensible au surapprentissage
   - ❌ Plus lent à entraîner
""")

print(f"\n🎯 CONCLUSION: Le modèle {best_model_name} offre le meilleur compromis")
print("   entre performance prédictive et généralisation.")

In [ ]:
# Sauvegarde du modèle final pour le déploiement
import shutil

# Le modèle final sera utilisé par l'API
final_model_path = '../ml_models_pkg/final_model.pkl'

if best_model_name == 'Ridge Regression':
    joblib.dump(ridge_model, final_model_path)
    joblib.dump(scaler, '../ml_models_pkg/final_scaler.pkl')
    model_type = 'ridge'
elif best_model_name == 'Random Forest':
    joblib.dump(rf_model, final_model_path)
    model_type = 'random_forest'
else:
    joblib.dump(gb_model, final_model_path)
    model_type = 'gradient_boosting'

# Sauvegarde des métadonnées
model_metadata = {
    'model_type': model_type,
    'model_name': best_model_name,
    'r2_score': results[best_idx]['R² Score'],
    'mae': results[best_idx]['MAE (t/ha)'],
    'rmse': results[best_idx]['RMSE (t/ha)'],
    'features': X_encoded.columns.tolist(),
    'target': 'yield',
    'training_samples': len(X_train),
    'test_samples': len(X_test)
}

joblib.dump(model_metadata, '../ml_models_pkg/model_metadata.pkl')

print(f"\n✅ Modèle final sauvegardé: {final_model_path}")
print(f"✅ Métadonnées sauvegardées: ../ml_models_pkg/model_metadata.pkl")
print(f"\n🚀 Le modèle est prêt pour le déploiement!")

In [ ]:
print(model_type)

## 7. Résumé pour le rapport

### Tableau récapitulatif des performances

| Modèle | R² Score | MAE (t/ha) | RMSE (t/ha) |
|--------|----------|------------|-------------|
| Ridge Regression | voir ci-dessus | voir ci-dessus | voir ci-dessus |
| Random Forest | voir ci-dessus | voir ci-dessus | voir ci-dessus |
| Gradient Boosting | voir ci-dessus | voir ci-dessus | voir ci-dessus |

### Interprétation des métriques

- **R² Score**: Proportion de la variance expliquée par le modèle (1 = parfait)
- **MAE**: Erreur moyenne en valeur absolue (en tonnes par hectare)
- **RMSE**: Erreur quadratique moyenne (pénalise plus les grosses erreurs)